# Install Essential Package

In [2]:


!pip install  tqdm
!pip install -U ultralytics
!pip install torch torchvision torchaudio
!pip install utils

import torch
import utils
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw ,ImageFont
import numpy as np
import matplotlib.pyplot as plt
import cv2


font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
random.seed(108)


In [1]:
ultralytics.checks()

NameError: name 'ultralytics' is not defined

# Show Label On Image
## Extract Info From Xml
## change PASCAL VOC .xml to yolov5 .txt

In [12]:
!pwd

/home/ray/StomataYolov8


In [13]:


dir_name = "AllData"



# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict['bboxes'] = []
    
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))

            info_dict['image_size'] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "robndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))
            info_dict['bboxes'].append(bbox)

    return info_dict


#xml choose complete

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , #
                           "hair":4} # human hair

# change robndbox to yolov5 format
def convert_to_yolov8(info_dict) :
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        # robndbox format
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = int(b["cx"])
        b_center_y = int(b["cy"])
        b_width = int(b["w"])
        b_height = int(b["h"])

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width    /= image_w
        b_height   /= image_h

        #Write the bbox details to the file
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))

    # Name of the file which we have to save
    save_file_name = os.path.join('./' + dir_name, info_dict["filename"] + '.txt' )

    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [14]:
# Get the annotationse
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./'+dir_name ) if x[-3:] == "xml"]


annotations.sort()
#print(annotations)
# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov8(info_dict)
    
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == "txt"]

print(annotations)

100%|██████████| 466/466 [00:00<00:00, 2139.57it/s]

['./AllData/T22L600_W1_A4_R4_P242_g178_2.txt', './AllData/T22L600_W1_A4_R7_P405_g26_2.txt', './AllData/T22L600_W3_GC1_R1_P13_g126_5.txt', './AllData/T16L600_W1_GC2_R6_P342_g203_1.txt', './AllData/T22L600_W1_A4_R4_P258_g150_4.txt', './AllData/T16L600_W1_GC1_R2_P123_g35_7.txt', './AllData/32TL600_W1_GC1_R1_P69_G92_6.txt', './AllData/T16L600_W1_GC1_R2_P93_g49_7.txt', './AllData/T22L600_W1_GC1_R2_P112_g103_8.txt', './AllData/32TL600_W1_GC1_R2_P94_G176_2.txt', './AllData/T22L600_W1_GC1_R2_P102_g97_4.txt', './AllData/T16L600_W3_GC1_R2_P92_g12_6.txt', './AllData/T16L600_W3_GC1_R2_P74_g55_4.txt', './AllData/T22L600_W1_A4_R4_P224_g109_8.txt', './AllData/32TL600_W1_GC1_R1_P55_G55_4.txt', './AllData/T16L600_W3_GC1_R2_P92_g12_3.txt', './AllData/T16L600_W3_GC1_R2_P97_g47_1.txt', './AllData/T22L600_W1_GC1_R2_P122_g13_4.txt', './AllData/T16L600_W3_GC2_R6_P382_g190_6.txt', './AllData/T16L600_W1_GC2_R6_P352_g193_5.txt', './AllData/T22L600_W1_GC1_R2_P121_g29_7.txt', './AllData/T16L600_W3_GC2_R6_P338_g17

# Testing the annotations

## Label the class on the image
Just for a sanity check, let us now test some of these transformed annotations. We randomly load one of the annotations and plot boxes using the transformed annotations, and visually inspect it to see whether our code has worked as intended.

Run the next cell multiple times. Every time, a random annotation is sampled.

Use command mv ./T22/*_label.jpg ./T22Label  
move label image to folder

In [15]:
random.seed(0)


class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

fnt = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 40)



def PlotAndSavebounding_box(image_name,image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size

    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h

    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]

    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)),outline ="red", width = 3)
        plotted_image.text((x0, y0-10), class_id_to_name_mapping[(int(obj_cls))] , fill = (255,0,255,255), font = fnt)
    
    # covert PIL image to cv2 image
    open_cv_image = np.array(image)

    # Convert RGB to BGR
    open_cv_image = open_cv_image[:, :, ::-1].copy()

    print(type(image))


    image_name = image_name.replace(".jpg", "_label.jpg")
    
    cv2.imwrite( image_name , open_cv_image, [cv2.IMWRITE_JPEG_QUALITY, 100])

    #plt.imshow(np.array(image))
    #plt.show()


# Get any random annotation file
#annotation_file = annotations[0]

'''
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
'''    


# use %cd T22Label to locate folder

# Plot the Bounding Box
# Polt all the Images Labels and annotations
for i in range(0,len(annotations)):

    annotation_file = annotations[i]

    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    #Get the corresponding image file
    image_file = annotation_file.replace("txt", "jpg")
    
    assert os.path.exists(image_file)

    #Load the image

    image = Image.open(image_file)
    PlotAndSavebounding_box(image_file,image, annotation_list)



<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.Jpeg

### Count all class of images
export the csv file of the dataset

In [40]:

annotations = [os.path.join('./'+dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == "txt"]

import pandas as pd


class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))




def CountAndSaveInstance(dic, annotation_list) :
    annotations = np.array(annotation_list)
    

    
    transformed_annotations = np.copy(annotations)
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        dic["count"][int(obj_cls)] += 1
    
    
# Set original  
dic = { "class" : class_name_to_id_mapping.keys() , "count" : [0,0,0,0,0]}



for i in range(0,len(annotations)):

    annotation_file = annotations[i]

    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    #
    CountAndSaveInstance(dic,annotation_list)


In [42]:
dic_df = pd.DataFrame(dic)
dic_df

,class,count
0,complete,862
1,incomplete,62
2,blurry.complete,587
3,blurry.incomplete,120
4,hair,874


In [43]:
dic_df.to_csv('T22Count.csv')

# Read Image and Annotation
###   Split to train datasets and Test data Set

In [16]:
# Read images and annotations
images = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == 'jpg' ]
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

## Create the folders to keep the splits.

In [19]:

path = './' + dir_name
#!mkdir ./T32/images ./T32/labels

os.mkdir(path + '/images')
os.mkdir(path + '/labels')



In [20]:
#!mkdir ./T32/images/train ./T32/images/val ./T32/images/test  ./T32/labels/train  ./T32/labels/val  ./T32/labels/test

os.mkdir(path + '/images/train')
os.mkdir(path + '/images/val')
os.mkdir(path + '/images/test')
os.mkdir(path + '/labels/train')
os.mkdir(path + '/labels/val')
os.mkdir(path + '/labels/test')

## Move the files to their respective folders

In [21]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False


# Move the splits into their folders
move_files_to_folder(train_images, path  + '/images/train')
move_files_to_folder(val_images, path + '/images/val/')
move_files_to_folder(test_images,  path +  '/images/test/')
move_files_to_folder(train_annotations, path + '/labels/train/')
move_files_to_folder(val_annotations, path + '/labels/val/')
move_files_to_folder(test_annotations, path + '/labels/test/')

##### Create a new file called road_sign_data.yaml and place it in the yolov5/data folder. Then populate it with the following.

train : ./T22/images/train/
val : ./T22/images/val/
test : ./T22/images/test/


##### number of classes
nc: 5

##### class names
names : ["complete", "incomplete","blurry.complete","blurry.incomplete" , "hair"]

# Train the Yolov8 Model

In [9]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 6.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.232
    Uninstalling ultralytics-8.0.232:
      Successfully uninstalled ultralytics-8.0.232


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8m-obb.pt")


/home/ray/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolov8s.pt")  # load an official model
results = model.train(data="data_All.yaml", epochs=250)
# Validate the model
results = model.val(data="data_All.yaml")  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.1.0 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data_All.yaml, epochs=250, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /home/ray/datasets/AllData/labels/train... 372 images, 0 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:00<00:00, 1021.52it/s]

train: New cache created: /home/ray/datasets/AllData/labels/train.cache



val: Scanning /home/ray/datasets/AllData/labels/val... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<00:00, 3116.08it/s]

val: New cache created: /home/ray/datasets/AllData/labels/val.cache


...


110 epochs completed in 0.185 hours.
Optimizer stripped from /home/ray/runs/detect/train8/weights/last.pt, 22.5MB
Optimizer stripped from /home/ray/runs/detect/train8/weights/best.pt, 22.5MB

Validating /home/ray/runs/detect/train8/weights/best.pt...
Ultralytics YOLOv8.1.0 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.13it/s]


                   all         47       1743      0.731      0.763      0.798      0.568
              complete         47        558      0.813      0.835      0.888      0.752
            incomplete         47         51      0.707      0.765      0.802      0.622
       blurry.complete         47        442      0.714      0.775      0.786      0.604
     blurry.incomplete         47        104      0.648       0.74      0.752      0.506
                  hair         47        588      0.773        0.7       0.76      0.359
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ray/runs/detect/train8
Ultralytics YOLOv8.1.0 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/ray/datasets/AllData/labels/val.cache... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.08s/it]


                   all         47       1743      0.732      0.763      0.799      0.566
              complete         47        558      0.814      0.833      0.889      0.754
            incomplete         47         51      0.707      0.765      0.802      0.609
       blurry.complete         47        442      0.713      0.774      0.787      0.602
     blurry.incomplete         47        104      0.652       0.74      0.754      0.503
                  hair         47        588      0.776      0.701      0.763       0.36
Speed: 0.2ms preprocess, 32.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /home/ray/runs/detect/train82


In [4]:
model = YOLO("/home/ray/runs/detect/T22+T32Train/weights/best.pt")  # load an official model 
results = model.val(data="data_T22T32.yaml")  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.232 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)


Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/ray/datasets/T22+T32/labels/val.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.02it/s]


                   all         17        829      0.775      0.792       0.82      0.603
              complete         17        212      0.768      0.882      0.801      0.687
            incomplete         17         32      0.753      0.688      0.835      0.614
       blurry.complete         17        198      0.811      0.773      0.774      0.643
     blurry.incomplete         17         24      0.674      0.792      0.794      0.561
                  hair         17        363       0.87      0.826      0.895       0.51
Speed: 0.2ms preprocess, 7.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/ray/runs/detect/val4


### Predict

In [24]:
from ultralytics import YOLO

model = YOLO("/home/ray/runs/detect/T16T22T32Train/weights/best.pt")  # load a custom model

# Predict with the model
results = model.predict( '/home/ray/Downloads/Stomata/All',save=True,save_crop = False,save_txt = True,save_conf = True)  # predict on an image


image 1/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P100_g103_2.jpg: 480x640 6 completes, 11 blurry.completes, 1 hair, 99.9ms
image 2/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P100_g103_6.jpg: 480x640 10 completes, 21 blurry.completes, 10 hairs, 3.7ms
image 3/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P106_g60_2.jpg: 480x640 18 completes, 14 blurry.completes, 1 hair, 3.6ms
image 4/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P106_g60_8.jpg: 480x640 8 completes, 25 blurry.completes, 5 hairs, 4.1ms
image 5/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P107_g54_4.jpg: 480x640 14 completes, 7 blurry.completes, 3.4ms
image 6/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P107_g54_8.jpg: 480x640 16 completes, 15 blurry.completes, 5 hairs, 3.9ms
image 7/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1_R2_P110_g88_4.jpg: 480x640 17 completes, 6 blurry.completes, 6 hairs, 3.4ms
image 8/300 /home/ray/Downloads/Stomata/All/T16L600_W1_GC1

##### Analysis Yolov8n  and Groundtruth predict  

Compare all classes count

In [10]:
import pandas as pd
import os
import numpy as np

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

#get predicts data
predicts = [os.path.join('/home/ray/runs/detect/predict2/labels', x) for x in os.listdir('/home/ray/runs/detect/predict2/labels') if x[-3:] == 'txt' ]


def GetPredictResults(predicts,dic):

    
    for i in range(0,len(predicts)):
        predict_file = predicts[i]
        with open(predict_file, "r") as file:
                predict_list = file.read().split("\n")[:-1]
                predict_list = [x.split(" ") for x in predict_list]
                predict_list = [[float(y) for y in x ] for x in predict_list]

        
        predict_list = np.array(predict_list)
    

        for ann in predict_list:
            obj_cls, x0, y0, x1, y1 , conf = ann
            dic["count"][int(obj_cls)] += 1
        
        

dic = { "class" : class_name_to_id_mapping.keys() , "count" : [0,0,0,0,0]}

GetPredictResults(predicts,dic)

dic_df = pd.DataFrame(dic)

dic_df.to_csv('T32PredictResults.csv')

dic_df



,class,count
0,complete,2044
1,incomplete,7
2,blurry.complete,1173
3,blurry.incomplete,19
4,hair,2809
